# 03 Tree Models and Feature Importance

Tree classifiers and feature importance vs permutation.


## Table of Contents
- Fit tree model
- Compare metrics
- Interpret importance


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Matching Guide
- `docs/guides/03_classification/03_tree_models_and_importance.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/03_classification/03_tree_models_and_importance.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Compare a tree-based classifier to logistic regression; interpret feature importance.


## Your Turn: Fit a tree model


In [ ]:
# TODO: Fit RandomForestClassifier (or GradientBoostingClassifier)
# Compare metrics to logistic regression
...


## Your Turn: Importance


In [ ]:
# TODO: Compare built-in feature importances vs permutation importance
# Discuss why they can disagree
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Fit tree model</summary>

```python
from sklearn.ensemble import RandomForestClassifier
from src import evaluation

rf = RandomForestClassifier(n_estimators=300, random_state=0)
rf.fit(X_tr, y_tr)
p_rf = rf.predict_proba(X_te)[:,1]
evaluation.classification_metrics(y_te.to_numpy(), p_rf)
```

</details>

<details><summary>Solution: Compare metrics</summary>

```python
# Compare to logistic metrics from earlier.
```

</details>

<details><summary>Solution: Interpret importance</summary>

```python
import pandas as pd
from sklearn.inspection import permutation_importance

imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
imp.head(10)

perm = permutation_importance(rf, X_te, y_te, n_repeats=10, random_state=0)
perm_imp = pd.Series(perm.importances_mean, index=X.columns).sort_values(ascending=False)
perm_imp.head(10)
```

</details>

